In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 18 03:56:49 2023

@author: Danyal
"""
import numpy as np
from IPython.display import Markdown
from sympy import *
from sympy.abc import x, y, h
init_printing()

In [ ]:
y = Function('x')
yxi, yxi1, yxi2= symbols('y_xi y_x(i+1) y_x(i-1)')
xi, h= symbols('x_i h')
wi, wif, wib= symbols('omega_i omega_(i+1) omega_(i-1)')

In [ ]:
ypp = (wif-2*wi+wib)/(h**2)
yp = (wif-wib)/(2*h)
y=wi
#display(ypp)
#display(yp)

In [12]:
equ = ypp-y*xi**2

boundary = (1,2)
f_bound  = (0.155,1.12)
step = 0.25
err = 1e-4

equ = sympify(equ)
equ = equ.subs(h,step)
display(equ)

                                 2          
16.0⋅ω₍ᵢ₊₁₎ + 16.0⋅ω₍ᵢ₋₁₎ - ωᵢ⋅xᵢ  - 32.0⋅ωᵢ

In [13]:
x_vals = np.linspace(boundary[0],boundary[1],int((boundary[1]-boundary[0])/step)+1)

c=0
n_list=list()
for i in x_vals:
    n = str(c)
    globals()['w%s' % n] = symbols(f'omega_{n}')
    globals()['x%s' % n] = symbols(f'x_{n}')
    globals()['eq_x%s' % n] = Eq(globals()['x%s' % n], i)
    display(globals()['eq_x%s' % n])
    globals()['x%s' % n] = S(i)
    n_list.append(c)
    c+=1

x₀ = 1.0

x₁ = 1.25

x₂ = 1.5

x₃ = 1.75

x₄ = 2.0

In [14]:
w_initial = f_bound[0]+0.1
globals()['w_%s' % 0] = f_bound[0]
globals()['w_%s' % n_list[-1]] = f_bound[1]

ws = list()
for n in n_list[1:-1]:
    globals()['w_%s' % n] = w_initial
    ws.append(globals()['w_%s' % n])
ws = Matrix(ws)

ws_all =list()
ws_all.append(ws)

for i in n_list[1:-1]:
    globals()['fequ_%s' % str(i)] = equ.subs(
                 {wi:globals()['w%s' % str(i)],
                  wif:globals()['w%s' % str(i+1)],
                  wib:globals()['w%s' % str(i-1)],
                  xi:globals()['x%s' % str(i)]})
    globals()['eq_x%s' % i] = Eq(globals()['fequ_%s' % str(i)], 0)
    #display(globals()['eq_x%s' % i])
    globals()['fequ_%s' % str(i)] = globals()['fequ_%s' % str(i)].expand()
    globals()['eq_x%s' % i] = Eq(globals()['fequ_%s' % str(i)], 0)
    display(globals()['eq_x%s' % i])
    print('-'*70)

jac_diffs = list()
for eqi in n_list[1:-1]:
    globals()['equ_diff_%s' % str(eqi)] = list()
    for i in n_list[1:-1]:
        globals()['equ_diff_%s' % str(eqi)].append(diff(globals()['fequ_%s' % str(eqi)],
                                                        globals()['w%s' % str(i)]))
    jac_diffs.append(globals()['equ_diff_%s' % str(eqi)])

jac = Matrix(jac_diffs)

print('-'*70)
display(jac)
#print(jac)
print('-'*70)
print('-'*70)

for itr in range(15):
    w_sub = dict()
    for i in n_list:
        w_sub[globals()['w%s' % i]]=globals()['w_%s' % i]
    #display(w_sub)

    jac_sub = jac.subs(w_sub)
    #display(jac_sub)
    #print('-'*70)

    jac_inv =jac_sub.inv()
    #display(jac_inv)
    #print('-'*70)

    for i in n_list:
        w_sub[xi]=globals()['x%s' % str(i)]
    f_sub = list()
    for i in n_list[1:-1]:
        globals()['fsub_%s' % str(i)] = globals()['fequ_%s' % str(i)].subs(w_sub)
        globals()['fsub_%s' % str(i)] = globals()['fsub_%s' % str(i)].subs(xi,globals()['x%s' % str(i)])
        f_sub.append(globals()['fsub_%s' % str(i)])

    f_mat = Matrix(f_sub)
    #display(f_mat)
    #pprint('-'*70)

    mat_prod = jac_inv*f_mat
    ws_out = ws - mat_prod
    wsp = ws
    ws=ws_out

    for i in n_list[1:-1]:
        globals()['w_%s' % i]=ws[i-1,0]

    ws_all.append(ws)
    display(ws_out)

    err_arr = wsp-ws
    err_arr = np.array(err_arr).reshape(1,-1)[0]
    err_arr = np.abs(err_arr)
    if len(np.where(err_arr<1e-5)[0])==len(n_list[1:-1]):
        break

    print('-'*70)
    print('-'*70)

16.0⋅ω₀ - 33.5625⋅ω₁ + 16.0⋅ω₂ = 0

----------------------------------------------------------------------


16.0⋅ω₁ - 34.25⋅ω₂ + 16.0⋅ω₃ = 0

----------------------------------------------------------------------


16.0⋅ω₂ - 35.0625⋅ω₃ + 16.0⋅ω₄ = 0

...

⎡-33.5625   16.0      0    ⎤
⎢                          ⎥
⎢  16.0    -34.25    16.0  ⎥
⎢                          ⎥
⎣   0       16.0   -35.0625⎦

----------------------------------------------------------------------
----------------------------------------------------------------------


⎡0.304828332362006⎤
⎢                 ⎥
⎢0.484425056556239⎥
⎢                 ⎥
⎣0.732144054328694⎦

----------------------------------------------------------------------
----------------------------------------------------------------------


⎡0.304828332362006⎤
⎢                 ⎥
⎢0.48442505655624 ⎥
⎢                 ⎥
⎣0.732144054328694⎦

In [ ]:
print('-'*70)
for m in ws_all:
    display(m)
    print('-'*70)

----------------------------------------------------------------------


⎡0.255⎤
⎢     ⎥
⎢0.255⎥
⎢     ⎥
⎣0.255⎦

----------------------------------------------------------------------


⎡0.436422588099365⎤
⎢                 ⎥
⎢0.709965337954939⎥
⎢                 ⎥
⎣0.949755921432698⎦

----------------------------------------------------------------------


⎡0.436422588099365⎤
⎢                 ⎥
⎢0.709965337954939⎥
⎢                 ⎥
⎣0.949755921432698⎦

----------------------------------------------------------------------
